Tensorflow GPU Check

In [1]:
import tensorflow as tf

strategy = tf.distribute.MirroredStrategy(devices = ["/gpu:0", "/gpu:1", "/gpu:2", "/gpu:3"], cross_device_ops = tf.distribute.HierarchicalCopyAllReduce())

with strategy.scope():
    x = tf.Variable(1.)
    
x

2021-11-22 12:02:33.477614: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]

In [4]:
print(tf.__version__)

2.3.0


--------

MEED2 emotion/intent distribution check.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import logging
import random
from datasets import *
import csv
import ast
from os.path import join

# # create the logging file
# logging.basicConfig(filename = "log/xenbot-os-dramatic-rest-red+_mask_.log", level = logging.INFO)
%matplotlib inline

2021-12-14 00:08:08.803043: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [7]:
# counting the emotion label
temp = np.load('datasets/red+/train/uttr_emots.npy')

np.array(np.unique(temp, return_counts=True)).T

array([[    0, 16441],
       [    1, 30014],
       [    2, 23627],
       [    3,  3664],
       [    4,  7886],
       [    5, 43936],
       [    6, 54646],
       [    7, 13908],
       [    8, 11225],
       [    9, 19436],
       [   10, 27694],
       [   11, 18776],
       [   12,  9406],
       [   13, 14791],
       [   14,  3683],
       [   15, 26167],
       [   16, 10487],
       [   17, 15207],
       [   18, 11427],
       [   19, 23463],
       [   20,  8147],
       [   21, 20527],
       [   22,  7862],
       [   23, 61770],
       [   24,  9171],
       [   25,  5102],
       [   26,  9157],
       [   27, 67943],
       [   28, 12794],
       [   29, 12837],
       [   30, 15401],
       [   31, 29117],
       [   32, 51166],
       [   33, 33866],
       [   34, 10056],
       [   35, 19832],
       [   36, 27533],
       [   37, 28282],
       [   38, 85438],
       [   39, 24639],
       [   40, 46876]])

In [2]:
# epoch 3 (lowest)
temp = pd.read_csv('emotions/model-red+/meed2+/red+_test_int.csv')

temp['meed2'].value_counts()

38    44063
32     8447
33     8117
40     4277
39     2369
36     2320
5       865
17      704
23      412
27      269
12      251
2       243
15      227
6       152
34      136
7        82
35       72
1        68
0        62
28       58
21       55
31       54
20       53
9        50
26       45
24       35
37       29
18       21
10       19
19       17
8        13
30        8
4         8
11        7
25        7
13        6
3         3
22        2
16        2
29        1
Name: meed2, dtype: int64

In [3]:
temp = pd.read_csv('emotions/model-red+/meed2+/red+_mask_test_int.csv')

temp['meed2'].value_counts()

38    44063
32     8446
33     8116
40     4277
39     2369
36     2320
5       866
17      704
23      412
27      270
12      251
2       243
15      227
6       152
34      136
7        82
35       72
1        68
0        62
28       58
21       55
31       54
20       53
9        50
26       45
24       35
37       29
18       21
10       19
19       17
8        13
30        8
4         8
11        7
25        7
13        6
3         3
22        2
16        2
29        1
Name: meed2, dtype: int64

In [2]:
red_plain = pd.read_csv('prediction-listener/red/plain/plain-os-dramatic-rest-red-plain.csv')

red_plain

,context,pred_y,tar_y
0,"--> disappointed I actually made good friends,...",sympathizing I’m sorry you’re going through th...,questioning What would you have to do to take ...
1,--> embarrassed This is one of those typical g...,sympathizing I’m so sorry you’re going through...,"neutral nah, you're awesome. although it may s..."
2,--> furious Seriously. You blocked your ex for...,sympathizing I’m so sorry you’re going through...,acknowledging Sounds to me like you just remem...
3,"--> trusting Just a note, I'm posting this for...",sympathizing I’m so sorry you’re going through...,suggesting you should tell them to post on red...
4,--> faithful I just feel like we never win. Sc...,wishing Good luck!,suggesting <URL> What's up with that?
...,...,...,...
495,--> annoyed Would anyone on here actually care...,sympathizing I’m here if you need someone to t...,questioning I don’t really know how to help bu...
496,--> suggesting I want to disappear and heal an...,apprehensive /<SUBREDDIT>,questioning Have you looked into yoga retreats?
497,--> anxious This has been happening a lot rece...,questioning Have you talked to your doctor abo...,anxious Whenever I get this panic feeling I ju...
498,--> suggesting Maybe I should just be dead. Ma...,suggesting Maybe I should just be dead. Maybe ...,apprehensive You need to work on your mindset ...


In [31]:
temp = red_plain.iloc[0, :]

temp['pred_y']

'sympathizing I’m sorry you’re going through this. If you need someone to talk to, feel free to message me.'

In [24]:
temp['context'].split('-->')

['',
 ' hopeful I really hope I can do this! I can at least try!\n',
 ' confident Go crush it!\n',
 " hopeful I did it!!!!! The manager said she'd call me back within the week!! Thank you for the support!"]

In [33]:
def extract_context_emotion(context):
    context_uttr_list = context.split('-->')
    emotion_list = []
    refined_uttr_list = []

    for uttr in context_uttr_list[1:]:
        emotion = uttr.split()[0]
        refined_uttr = ' '.join(uttr.split()[1:])
        emotion_list.append(emotion)
        refined_uttr_list.append(refined_uttr)
    
    return emotion_list, refined_uttr_list

def extract_pred_emotion(pred):
    emotion = pred.split()[0]
    uttr = ' '.join(pred.split()[1:])
    
    return emotion, uttr

def extract_tar_emotion(tar):
    emotion = tar.split()[0]
    uttr = ' '.join(tar.split()[1:])
    
    return emotion, uttr

def build_emotion_df(df):
    context_emotion_col = []
    context_col = []
    pred_emotion_col, tar_emotion_col = [], []
    pred_y_col, tar_y_col = [], []
    
    for i, row in df.iterrows():
        context_emotion, context = extract_context_emotion(row['context'])
        pred_emotion, pred_y = extract_pred_emotion(row['pred_y'])
        tar_emotion, tar_y = extract_tar_emotion(row['tar_y'])
        
        context_emotion_col.append(context_emotion)
        context_col.append(context)
        pred_emotion_col.append(pred_emotion)
        pred_y_col.append(pred_y)
        tar_emotion_col.append(tar_emotion)
        tar_y_col.append(tar_y)
    
    return pd.DataFrame({'emotion': context_emotion_col, 'context': context_col, 
                         'pred_emotion': pred_emotion_col, 'pred_y': pred_y_col, 
                         'tar_emotion': tar_emotion_col, 'tar_y': tar_y_col})

In [43]:
build_emotion_df(red_plain)

,emotion,context,pred_emotion,pred_y,tar_emotion,tar_y
0,[disappointed],"[I actually made good friends, hung out with p...",sympathizing,I’m sorry you’re going through this. If you ne...,questioning,What would you have to do to take a year off a...
1,[embarrassed],"[This is one of those typical girl stories, wh...",sympathizing,I’m so sorry you’re going through this. I hope...,neutral,"nah, you're awesome. although it may seem a bu..."
2,[furious],[Seriously. You blocked your ex for a reason. ...,sympathizing,I’m so sorry you’re going through this.,acknowledging,Sounds to me like you just remembered why you ...
3,[trusting],"[Just a note, I'm posting this for a friend wh...",sympathizing,I’m so sorry you’re going through this. I hope...,suggesting,you should tell them to post on reddit rather ...
4,[faithful],[I just feel like we never win. Screw the mach...,wishing,Good luck!,suggesting,<URL> What's up with that?
...,...,...,...,...,...,...
495,[annoyed],[Would anyone on here actually care if there i...,sympathizing,I’m here if you need someone to talk to.,questioning,"I don’t really know how to help but, wanna talk?"
496,[suggesting],[I want to disappear and heal and move on from...,apprehensive,/<SUBREDDIT>,questioning,Have you looked into yoga retreats?
497,[anxious],"[This has been happening a lot recently, but I...",questioning,Have you talked to your doctor about this?,anxious,Whenever I get this panic feeling I just pick ...
498,[suggesting],[Maybe I should just be dead. Maybe I’ll be ha...,suggesting,Maybe I should just be dead. Maybe I’ll be hap...,apprehensive,You need to work on your mindset first. Are th...


In [ ]:
# epoch 2
red_test = pd.read_csv('emotions/model-red/meed2/red+_test_int.csv')
red_test['meed2'].value_counts()

In [ ]:
red_test['ground'].value_counts()

In [ ]:
with open('datasets/red+_mask/test/encoded.txt') as infile:
    lines = infile.readlines()
    
    for line in lines:
        print(line.split('\t')[-1])

In [5]:
speaker_indices, listener_indices = [], []
input_line_indices = []

with open('datasets/red+_mask/test/encoded.txt') as infile:
    lines = infile.readlines()
    line_index = 0
    
    for line in lines:
        line_lst = line.split('\t')
        if len(line_lst) < 5:
            print(line)
            print(line_index)
            
        start_ind, end_ind = int(line_lst[-1].split(',')[0]), int(line_lst[-1].split(',')[1])

        if (start_ind + end_ind) % 2 == 1:
            speaker_indices.append(start_ind)
            listener_indices.append(end_ind)
            input_line_indices.append(line_index)

        line_index += 1

In [12]:
test_indices = random.sample(input_line_indices, 500)
test_indices.sort()
test_indices

[71,
 198,
 240,
 241,
 395,
 494,
 524,
 727,
 729,
 897,
 1040,
 1208,
 1501,
 1592,
 1752,
 1776,
 2042,
 2236,
 2717,
 2753,
 2834,
 2920,
 2996,
 3076,
 3454,
 3544,
 3643,
 3710,
 3809,
 3873,
 3955,
 4085,
 4092,
 4472,
 4841,
 4949,
 4960,
 5091,
 5179,
 5180,
 5295,
 5392,
 5639,
 5679,
 5738,
 5783,
 5910,
 6292,
 6593,
 6718,
 7215,
 7237,
 7248,
 7429,
 7457,
 7822,
 7943,
 7956,
 8019,
 8086,
 8170,
 8234,
 8246,
 8343,
 8356,
 8545,
 8551,
 8616,
 8908,
 8985,
 9232,
 9271,
 9292,
 9500,
 9520,
 9524,
 9647,
 9804,
 9829,
 10157,
 10297,
 10319,
 10355,
 10394,
 10610,
 10698,
 10808,
 10896,
 10916,
 10945,
 11037,
 11214,
 11316,
 11567,
 11691,
 11840,
 11846,
 11866,
 11930,
 12087,
 12620,
 13224,
 13787,
 13816,
 13998,
 14088,
 14211,
 14321,
 14582,
 14716,
 14719,
 14744,
 14768,
 14882,
 14967,
 15062,
 15125,
 15144,
 15252,
 15327,
 15575,
 15684,
 15693,
 15978,
 15992,
 16167,
 16176,
 16302,
 16577,
 16600,
 16664,
 16674,
 16773,
 16861,
 16921,
 16924,
 1

In [5]:
temp = np.load('prediction-listener/red+_mask.npy')[127]

np.save('prediction-listener/red+_mask_16302.npy', temp)

In [13]:
np.save('prediction-listener/red+_mask.npy', test_indices)

-------

Merge the training dataframe with the annotated rationale masks.

In [3]:
train_output = pd.read_csv('empathy-mental-health/dataset/train_output.csv')
train_df_short = pd.read_csv('transformer models/datasets/red/train_df_short.csv')

/root/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [20]:
def add_ER_IP_EX(convs_df, sl_ori):
    '''
    Add the columns of ER(emotional reaction), IP(interpretation), EX(exploration) and their rationale masks
    to the original dataframe.
    '''
    convs, sl = convs_df.copy(), sl_ori.copy()
    sl.rename({'response_post': 'text'}, axis = 1, inplace = True)

    feat_list_to_keep = ['text', 'ER_label', 'ER_rationale_mask', 'IP_label', 'IP_rationale_mask', 'EX_label', 'EX_rationale_mask']
    convs_masked = pd.merge(convs, sl[feat_list_to_keep], how = 'left', on = 'text')
    
    # fill in the missing value
    # for comm level just fill 0; for masks fill the list with length as the text's
    convs_masked['ER_label'].fillna(0, inplace = True)
    convs_masked['IP_label'].fillna(0, inplace = True)
    convs_masked['EX_label'].fillna(0, inplace = True)

    for row in convs_masked.itertuples():

        if pd.isna(row.ER_rationale_mask):
            convs_masked.loc[row.Index, 'ER_rationale_mask'] = str([1] * len(row.text.split()))
        
        if pd.isna(row.IP_rationale_mask):
            convs_masked.loc[row.Index, 'IP_rationale_mask'] = str([1] * len(row.text.split()))
            
        if pd.isna(row.EX_rationale_mask):
            convs_masked.loc[row.Index, 'EX_rationale_mask'] = str([1] * len(row.text.split()))
            
    
    return convs_masked

In [ ]:
# update by adding the columns
train_df_masked = add_ER_IP_EX(train_df_short, train_output)

train_df_masked.to_csv('transformer models/datasets/red/train_df_masked.csv', index = False)

In [2]:
train_df_masked = pd.read_csv('datasets/red/train_df_masked.csv')

train_df_masked

/root/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,conversation id,subreddit,post title,author,dialog turn,text,compound,sentiment,emotion prediction,ER_label,ER_rationale_mask,IP_label,IP_rationale_mask,EX_label,EX_rationale_mask
0,1,depression,"I've tried everything, but it's back.",speaker,1,It got worse and worse. I know they love me bu...,0.9948,positive,ashamed,0.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,1,depression,"I've tried everything, but it's back.",listener_1,2,Hey you seem like a really sweet and nice pers...,0.8341,positive,questioning,1.0,"[0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...",0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...",2.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,4,depression,List of strange things I find depressing- Anyo...,speaker,1,* Sundays * Yellow sunlight * Homework * Leav...,0.1786,positive,lonely,0.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,4,depression,List of strange things I find depressing- Anyo...,listener_1,2,1.rotten fruit 2.my messy hair 3.children 4.fa...,-0.3612,negative,annoyed,0.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,7,depression,Fuck me,speaker,1,I want to fucking die,-0.5984,negative,ashamed,0.0,"[1, 1, 1, 1, 1]",0.0,"[1, 1, 1, 1, 1]",0.0,"[1, 1, 1, 1, 1]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1011611,1494877,MentalHealthSupport,Childhood Abuse Survivor turned Trauma Informe...,listener_1,2,Thankyou for this. For sharing your personal e...,0.4753,positive,grateful,1.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1011612,1494877,MentalHealthSupport,Childhood Abuse Survivor turned Trauma Informe...,speaker,3,It is my absolute honour and privilege to be a...,0.9098,positive,grateful,0.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1011613,1494880,MentalHealthSupport,My team and I made this simple chatbot that he...,speaker,1,My team and I made this simple chatbot that he...,0.9117,positive,confident,0.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1011614,1494880,MentalHealthSupport,My team and I made this simple chatbot that he...,listener_1,2,Thanks for this,0.4404,positive,acknowledging,0.0,"[1, 1, 1]",0.0,"[1, 1, 1]",0.0,"[1, 1, 1]"


In [6]:
train_df_masked = train_df_masked.drop_duplicates()
train_df_masked.to_csv('datasets/red/train_df_masked.csv', index = False)

---------
Check if testing set works suitably.

In [3]:
from pytorch_transformers import RobertaTokenizer
from datasets import *
from math import ceil

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
data_path = 'datasets/red'
batch_size = 512
buffer_size = 100000
max_length = 100
train_dataset, val_dataset, test_dataset = create_datasets(tokenizer, data_path, buffer_size, batch_size, max_length, contain_comm = False, contain_mask = False)

Vocabulary size is 50265.
Reading encoded data from "datasets/red/train/encoded.txt"...
Reading emotions from "datasets/red/train/uttr_emots.npy"...


100%|████████████████████████████████| 525657/525657 [00:33<00:00, 15584.32it/s]
2021-11-24 20:29:18.029337: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-11-24 20:29:18.209389: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:04:00.0 name: NVIDIA TITAN X (Pascal) computeCapability: 6.1
coreClock: 1.531GHz coreCount: 28 deviceMemorySize: 11.91GiB deviceMemoryBandwidth: 447.48GiB/s
2021-11-24 20:29:18.210733: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 1 with properties: 
pciBusID: 0000:05:00.0 name: NVIDIA TITAN X (Pascal) computeCapability: 6.1
coreClock: 1.531GHz coreCount: 28 deviceMemorySize: 11.91GiB deviceMemoryBandwidth: 447.48GiB/s
2021-11-24 20:29:18.212072: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 2 with properties: 
pciBusID: 0000:84:00.0 name: NVIDIA TITAN X (Pascal) computeCapability: 6.1
core

Reading encoded data from "datasets/red/valid/encoded.txt"...
Reading emotions from "datasets/red/valid/uttr_emots.npy"...


100%|██████████████████████████████████| 65714/65714 [00:04<00:00, 15664.65it/s]


Reading encoded data from "datasets/red/test/encoded.txt"...
Reading emotions from "datasets/red/test/uttr_emots.npy"...


100%|██████████████████████████████████| 13745/13745 [00:00<00:00, 16144.19it/s]


In [24]:
with mirrored_strategy.scope():
    for (batch, inputs) in enumerate(train_dataset):
        inp, inp_seg, inp_emot, tar_inp, tar_real, tar_seg, tar_emot = inputs
        break

why create_test_dataset fail to capture speaker/listener?

In [28]:
read_path = 'datasets/red+_mask/train' # this dataset has no comm files
max_length = 100

encoded_path = join(read_path, 'encoded.txt')
f = open(encoded_path, 'r', encoding = 'utf-8')
print('Reading encoded data from \"{}\"...'.format(encoded_path))
lines = f.read().splitlines()

uttr_emots_path = join(read_path, 'uttr_emots.npy')
# uttr_comms_path = join(read_path, 'uttr_comms.npy')
uttr_masks_path = join(read_path, 'uttr_masks.npy')
print('Reading emotions from \"{}\"...'.format(uttr_emots_path))
# print('Reading emotions from \"{}\"...'.format(uttr_comms_path))
print('Reading emotions from \"{}\"...'.format(uttr_masks_path))
uttr_emots = np.load(uttr_emots_path)
# uttr_comms = np.load(uttr_comms_path)
uttr_masks = np.load(uttr_masks_path, allow_pickle = True)
#uttr_emots = np.argsort(uttr_emots, axis = 1)

# RoBERTa uses 1 as the padding value
inputs = np.ones((len(lines), max_length), dtype = np.int32)
input_segments = np.ones((len(lines), max_length), dtype = np.int32)
input_emots = np.zeros((len(lines), max_length), dtype = np.int32)
# input_comms = np.zeros((len(lines), max_length), dtype = np.int32)
targets_i = np.ones((len(lines), max_length), dtype = np.int32)
targets_r = np.ones((len(lines), max_length), dtype = np.int32)
target_segments = np.ones((len(lines), max_length), dtype = np.int32)
target_emots = np.zeros(len(lines), dtype = np.int32)
targets_mask = np.zeros((len(lines), max_length), dtype = np.int32)
# target_comms = np.zeros(len(lines), dtype = np.int32)

Reading encoded data from "datasets/red+_mask/train/encoded.txt"...
Reading emotions from "datasets/red+_mask/train/uttr_emots.npy"...
Reading emotions from "datasets/red+_mask/train/uttr_masks.npy"...


In [29]:
n = 0
for i, line in tqdm(enumerate(lines), total = len(lines)):
    # 0,8976,2156,2085,52,32,11,761,9,10,910,1182,479,2 0,0,0,0,0,0,0,0,0,0,0,0,0,0 0,8976,2156,38,619,101,38,697,10,1256,455,301,479,38,1266,2156,38,128,119,1819,117,810,326,4218,479,256,119,479,2   1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1   0,1
    inp_str, inp_seg_str, tar_str, tar_seg_str, idx_str = line.split('\t')
    j, k = [int(s) for s in idx_str.split(',')]

    inp_ids = [int(s) for s in inp_str.split(',')]
    inp_seg_ids = [int(s) for s in inp_seg_str.split(',')]
    tar_ids = [int(s) for s in tar_str.split(',')]
    tar_seg_ids = [int(s) for s in tar_seg_str.split(',')]


    seg_id = 0
    for x in range(len(inp_seg_ids)):
        if inp_seg_ids[x] != seg_id:
            j += 1
            seg_id = inp_seg_ids[x]
        #input_emots[i,x] = uttr_emots[j,-1]
        input_emots[i,x] = uttr_emots[j] # add the current emotion label on the speaker segment indices in the input_emots array (2D with shape as (# of example, max_length))
    #target_emots[i] = uttr_emots[k,-1]
    target_emots[i] = uttr_emots[k] # simply add the emotion label to the target_emots array (1D with shape as (# of example, ))
    # target_comms[i] = uttr_comms[k]
    
    inputs[i,:len(inp_ids)] = inp_ids
    input_segments[i,:len(inp_seg_ids)] = inp_seg_ids
    targets_i[i,:len(tar_ids)-1] = tar_ids[:-1]
    targets_r[i,:len(tar_ids)-1] = tar_ids[1:]
    target_segments[i,:len(tar_seg_ids)-1] = tar_seg_ids[:-1]
    
    if len(uttr_masks[k]) > max_length:
        targets_mask[i,:max_length] = uttr_masks[k][:max_length]
    else:
        targets_mask[i,:len(uttr_masks[k])] = uttr_masks[k]
    

100%|████████████████████████████████| 241247/241247 [00:17<00:00, 13649.56it/s]


In [33]:
(targets_mask > 1) * 1

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

----

check the new version (adding comm level) work suitably.

In [4]:
# need to adjust the create_dataset in datasets.py to contain comm level
read_path = 'datasets/red+/test'
max_length = 100


create_dataset(read_path, max_length, contain_comm = True)

  0%|                                                             | 0/73629 [00:00<?, ?it/s]

Reading encoded data from "datasets/red+/test/encoded.txt"...
Reading emotions from "datasets/red+/test/uttr_emots.npy"...
Reading emotions from "datasets/red+/test/uttr_comms.npy"...


100%|██████████████████████████████████████████████| 73629/73629 [00:05<00:00, 12595.15it/s]


((<TensorSliceDataset shapes: (100,), types: tf.int32>,
  <TensorSliceDataset shapes: (100,), types: tf.int32>,
  <TensorSliceDataset shapes: (100,), types: tf.int32>,
  <TensorSliceDataset shapes: (100,), types: tf.int32>,
  <TensorSliceDataset shapes: (100,), types: tf.int32>,
  <TensorSliceDataset shapes: (100,), types: tf.int32>,
  <TensorSliceDataset shapes: (100,), types: tf.int32>,
  <TensorSliceDataset shapes: (), types: tf.int32>,
  <TensorSliceDataset shapes: (), types: tf.int32>),
 73629)